# Proyecto de Machine Learning: Clasificación de Planes de Megaline

## Introducción

La compañía móvil **Megaline** se enfrenta a un desafío importante: muchos de sus clientes siguen utilizando planes antiguos, mientras que la empresa ha lanzado dos nuevos planes más competitivos llamados **Smart** y **Ultra**. Nuestro objetivo en este proyecto es ayudar a Megaline a desarrollar un modelo que pueda analizar el comportamiento de sus clientes y recomendar el plan adecuado entre estos dos nuevos planes. 

Contamos con datos que incluyen el comportamiento mensual de los usuarios que ya han migrado a uno de estos dos planes. Usaremos este dataset para entrenar un modelo que, basándose en las características de uso, pueda predecir si un cliente debería estar en el plan **Smart** o en el plan **Ultra**.

## Descripción de los datos

El dataset **`users_behavior.csv`** contiene la siguiente información sobre el comportamiento mensual de los usuarios:

- **calls**: Número de llamadas realizadas por el usuario en el mes.
- **minutes**: Duración total de las llamadas en minutos.
- **messages**: Número de mensajes de texto enviados por el usuario.
- **mb_used**: Tráfico de Internet utilizado en megabytes (MB).
- **is_ultra**: Etiqueta del plan en el mes actual (Ultra - 1, Smart - 0).

## Objetivo

Nuestro objetivo es desarrollar un modelo de clasificación que pueda predecir con la mayor precisión posible si un cliente debería estar en el plan **Smart** o en el plan **Ultra**. El umbral de precisión requerido es de al menos **0.75**. 

Trabajaremos con diferentes modelos y ajustaremos los hiperparámetros para optimizar el rendimiento del modelo, usando conjuntos de datos divididos en entrenamiento, validación y prueba.


## Paso 1: Abrir y examinar el archivo de datos

En este primer paso, comenzaremos por abrir el conjunto de datos proporcionado **`users_behavior.csv`**, que contiene información sobre el comportamiento mensual de los usuarios de Megaline. La primera tarea será cargar los datos y examinarlos para entender la estructura del dataset y verificar que los datos sean apropiados para el análisis. Esto incluye observar el tamaño del conjunto de datos, las columnas disponibles y una primera mirada a las estadísticas básicas de las variables.

El objetivo es familiarizarnos con los datos y asegurarnos de que estén listos para ser procesados en los siguientes pasos.


In [2]:
# Importar las bibliotecas necesarias
import pandas as pd

# Cargar el archivo de datos
df = pd.read_csv('/datasets/users_behavior.csv')

# Ver las primeras filas del dataset
df.head()

# Mostrar información general sobre el dataset
df.info()

# Ver estadísticas básicas de las variables numéricas
df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


## Resultados de la inspección inicial de los datos

El dataset **`users_behavior.csv`** contiene un total de **3,214 observaciones** y **5 columnas** que describen el comportamiento de los usuarios de Megaline. Cada fila representa un registro mensual del comportamiento de un usuario, con las siguientes características:

- **calls**: Número de llamadas realizadas en el mes.
- **minutes**: Duración total de las llamadas en minutos.
- **messages**: Número de mensajes de texto enviados en el mes.
- **mb_used**: Tráfico de Internet utilizado en megabytes (MB).
- **is_ultra**: Etiqueta binaria que indica si el usuario utiliza el plan **Ultra** (1) o el plan **Smart** (0).

### Detalles importantes:
- El dataset **no tiene valores nulos**, lo que significa que no necesitaremos lidiar con datos faltantes, lo cual facilita el análisis.
- Las columnas **calls**, **minutes**, **messages**, y **mb_used** son de tipo `float64`, mientras que la columna **is_ultra** es de tipo `int64`, ya que representa una variable categórica binaria.
- Observamos que los valores de las variables **calls**, **minutes**, **messages**, y **mb_used** tienen diferentes rangos de variabilidad:
  - **calls**: Va de 0 a 244 llamadas por mes.
  - **minutes**: Va de 0 a 1632 minutos por mes.
  - **messages**: Va de 0 a 224 mensajes por mes.
  - **mb_used**: Va de 0 MB a 49,745 MB por mes.

### Conclusión referente a los datos:
Los datos están limpios y listos para ser segmentados en conjuntos de entrenamiento, validación y prueba. En el próximo paso, procederemos a esta segmentación para entrenar y evaluar nuestros modelos de clasificación.


## Paso 2: Segmentación del dataset en conjuntos de entrenamiento, validación y prueba

En este paso, segmentaremos los datos en tres conjuntos: **entrenamiento**, **validación** y **prueba**. Esta división es fundamental para evaluar la calidad de los modelos de manera efectiva.

- **Conjunto de entrenamiento**: Se utilizará para entrenar los modelos. Este conjunto contendrá la mayor parte de los datos.
- **Conjunto de validación**: Se utilizará para ajustar los hiperparámetros de los modelos y seleccionar el modelo con el mejor rendimiento.
- **Conjunto de prueba**: Una vez elegido el mejor modelo y ajustados los hiperparámetros, usaremos este conjunto para realizar la evaluación final del rendimiento del modelo. Esto nos permitirá determinar si el modelo generaliza bien a datos que no ha visto antes.

Vamos a dividir el dataset de la siguiente manera:
- **60%** de los datos para el conjunto de **entrenamiento**.
- **20%** de los datos para el conjunto de **validación**.
- **20%** de los datos para el conjunto de **prueba**.

Para realizar esta división, utilizaremos la función `train_test_split` de la librería **scikit-learn**, que nos permite realizar una división aleatoria de los datos.


In [3]:
# Importar las bibliotecas necesarias
from sklearn.model_selection import train_test_split

# Definir las características (features) y el objetivo (target)
features = df.drop(['is_ultra'], axis=1)  # Todas las columnas excepto 'is_ultra'
target = df['is_ultra']  # La columna objetivo es 'is_ultra'

# Dividir los datos en conjunto de entrenamiento (60%) y un conjunto temporal (40%) para validación y prueba
features_train, features_temp, target_train, target_temp = train_test_split(
    features, target, test_size=0.4, random_state=12345)

# Dividir el conjunto temporal en conjunto de validación (20%) y conjunto de prueba (20%)
features_valid, features_test, target_valid, target_test = train_test_split(
    features_temp, target_temp, test_size=0.5, random_state=12345)

# Imprimir el tamaño de cada conjunto para verificar la segmentación
print(f'Tamaño del conjunto de entrenamiento: {len(features_train)}')
print(f'Tamaño del conjunto de validación: {len(features_valid)}')
print(f'Tamaño del conjunto de prueba: {len(features_test)}')


Tamaño del conjunto de entrenamiento: 1928
Tamaño del conjunto de validación: 643
Tamaño del conjunto de prueba: 643


## Paso 3: Investigación de la calidad de diferentes modelos

En este paso, probaremos diferentes modelos de clasificación para identificar cuál ofrece el mejor rendimiento en términos de exactitud. Probaremos una variedad de algoritmos, ajustando sus hiperparámetros para mejorar la calidad de las predicciones.

El objetivo es encontrar el modelo que proporcione una precisión igual o superior a **0.75** en el conjunto de validación, que es el umbral que se requiere para este proyecto.

Los modelos que investigaremos incluyen:
1. **Árbol de Decisión (Decision Tree)**
2. **Bosque Aleatorio (Random Forest)**
3. **Regresión Logística (Logistic Regression)**

Para cada uno de estos modelos, ajustaremos los hiperparámetros relevantes y calcularemos la exactitud en el conjunto de validación. Luego, seleccionaremos el mejor modelo basándonos en su rendimiento.


In [4]:
# Importar las bibliotecas necesarias
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Definir un diccionario para almacenar los resultados de exactitud de los diferentes modelos
results = {}

# Modelo 1: ÁRBOL DE DECISIÓN
# Entrenamos un modelo de Árbol de Decisión
tree_model = DecisionTreeClassifier(random_state=12345)
tree_model.fit(features_train, target_train)
tree_predictions = tree_model.predict(features_valid)

# Calcular la exactitud del Árbol de Decisión
tree_accuracy = accuracy_score(target_valid, tree_predictions)
results['Árbol de Decisión'] = tree_accuracy
print(f"Exactitud del Árbol de Decisión: {tree_accuracy:.4f}")

# Modelo 2: BOSQUE ALEATORIO
# Entrenamos un modelo de Bosque Aleatorio
forest_model = RandomForestClassifier(random_state=12345, n_estimators=100)
forest_model.fit(features_train, target_train)
forest_predictions = forest_model.predict(features_valid)

# Calcular la exactitud del Bosque Aleatorio
forest_accuracy = accuracy_score(target_valid, forest_predictions)
results['Bosque Aleatorio'] = forest_accuracy
print(f"Exactitud del Bosque Aleatorio: {forest_accuracy:.4f}")

# Imprimir los resultados finales para comparar los modelos
print("\nResultados de exactitud en el conjunto de validación:")
for model, accuracy in results.items():
    print(f"{model}: {accuracy:.4f}")


Exactitud del Árbol de Decisión: 0.7138
Exactitud del Bosque Aleatorio: 0.7854

Resultados de exactitud en el conjunto de validación:
Árbol de Decisión: 0.7138
Bosque Aleatorio: 0.7854


In [5]:
# Modelo 3: REGRESIÓN LOGÍSTICA
# Entrenamos un modelo de Regresión Logística
log_model = LogisticRegression(random_state=12345, solver='liblinear')
log_model.fit(features_train, target_train)
log_predictions = log_model.predict(features_valid)

# Calcular la exactitud de la Regresión Logística
log_accuracy = accuracy_score(target_valid, log_predictions)
results['Regresión Logística'] = log_accuracy
print(f"Exactitud de la Regresión Logística: {log_accuracy:.4f}")

# Imprimir los resultados finales actualizados para comparar los tres modelos
print("\nResultados de exactitud en el conjunto de validación:")
for model, accuracy in results.items():
    print(f"{model}: {accuracy:.4f}")


Exactitud de la Regresión Logística: 0.7589

Resultados de exactitud en el conjunto de validación:
Árbol de Decisión: 0.7138
Bosque Aleatorio: 0.7854
Regresión Logística: 0.7589


## Resultados de la evaluación inicial de los modelos

Hasta este punto, hemos evaluado tres modelos diferentes de clasificación para recomendar un plan a los usuarios de Megaline: **Árbol de Decisión**, **Bosque Aleatorio**, y **Regresión Logística**. A continuación, se presentan los resultados de exactitud en el conjunto de validación para cada modelo:

- **Árbol de Decisión**: 0.7138
- **Bosque Aleatorio**: 0.7854
- **Regresión Logística**: 0.7589

### Análisis de los resultados:
- El **Árbol de Decisión** presentó el menor rendimiento, con una exactitud del 71.38%. Si bien el modelo ofrece una interpretación clara, no logra superar el umbral mínimo de exactitud de **0.75**.
- La **Regresión Logística** alcanzó una exactitud de **0.7589**, superando el umbral requerido. Este modelo es adecuado para clasificaciones binarias simples, como en este caso.
- Sin embargo, el **Bosque Aleatorio** ha sido el modelo más exitoso hasta ahora, con una exactitud de **0.7854**, lo que lo convierte en el mejor modelo de los tres evaluados.

### Próximo paso: Ajuste de hiperparámetros del Bosque Aleatorio
Para mejorar aún más el rendimiento del **Bosque Aleatorio**, ajustaremos dos de sus hiperparámetros más importantes:
1. **n_estimators**: El número de árboles en el bosque.
2. **max_depth**: La profundidad máxima de los árboles.

Realizaremos una búsqueda exhaustiva probando diferentes valores para estos hiperparámetros y seleccionaremos la mejor combinación basándonos en la exactitud obtenida en el conjunto de validación. El objetivo es superar el rendimiento actual de **0.7854** de exactitud.

Una vez completado este ajuste, seleccionaremos el modelo final para la evaluación en el conjunto de prueba.


In [6]:
# Ajuste de hiperparámetros del Bosque Aleatorio
best_accuracy = 0
best_estimators = 0
best_depth = 0

for n_estimators in range(50, 151, 50):  # Probar con diferentes números de árboles
    for max_depth in range(5, 16, 5):  # Probar con diferentes profundidades máximas
        forest_model = RandomForestClassifier(random_state=12345, n_estimators=n_estimators, max_depth=max_depth)
        forest_model.fit(features_train, target_train)
        forest_predictions = forest_model.predict(features_valid)
        accuracy = accuracy_score(target_valid, forest_predictions)
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_estimators = n_estimators
            best_depth = max_depth

# Imprimir el mejor modelo encontrado
print(f"Mejor exactitud: {best_accuracy:.4f} con n_estimators = {best_estimators} y max_depth = {best_depth}")


Mejor exactitud: 0.7947 con n_estimators = 100 y max_depth = 5


## Paso 4: Evaluación final del modelo en el conjunto de prueba

Después de ajustar los hiperparámetros del **Bosque Aleatorio** y seleccionar el mejor modelo, procederemos a evaluar su rendimiento final en el **conjunto de prueba**. Este conjunto contiene datos que el modelo no ha visto ni durante el entrenamiento ni durante la validación, lo que nos permitirá obtener una evaluación justa de cómo se comportará el modelo en situaciones del mundo real.

El objetivo es obtener una exactitud de al menos **0.75** en el conjunto de prueba. Dado que el modelo ha alcanzado una exactitud de **0.7947** en el conjunto de validación, esperamos que su rendimiento en el conjunto de prueba sea similar o mejor.


In [7]:
# Evaluación final del modelo en el conjunto de prueba

# Usamos el mejor modelo encontrado con n_estimators=100 y max_depth=5
final_model = RandomForestClassifier(random_state=12345, n_estimators=100, max_depth=5)
final_model.fit(features_train, target_train)

# Realizar predicciones en el conjunto de prueba
test_predictions = final_model.predict(features_test)

# Calcular la exactitud en el conjunto de prueba
test_accuracy = accuracy_score(target_test, test_predictions)

# Imprimir el resultado de la evaluación final
print(f"Exactitud del modelo en el conjunto de prueba: {test_accuracy:.4f}")


Exactitud del modelo en el conjunto de prueba: 0.7900


## Evaluación final del modelo en el conjunto de prueba

Después de haber ajustado los hiperparámetros y seleccionado el mejor modelo, evaluamos el **Bosque Aleatorio** con los parámetros óptimos en el **conjunto de prueba**. Este conjunto contiene datos que el modelo no ha visto antes, por lo que nos brinda una evaluación precisa de su rendimiento en situaciones del mundo real.

### Resultados de la evaluación final:
- **Exactitud del modelo en el conjunto de prueba**: 0.7900

### Análisis:
El modelo ha alcanzado una exactitud del **79.00%** en el conjunto de prueba, lo cual supera el umbral mínimo de **0.75** establecido para el proyecto. Esto indica que el modelo es confiable y puede hacer predicciones precisas sobre el plan que mejor se adapta a los usuarios de Megaline.

### Conclusión:
En resumen, después de evaluar varios modelos de clasificación, incluyendo el **Árbol de Decisión**, **Bosque Aleatorio**, y **Regresión Logística**, el **Bosque Aleatorio** demostró ser el modelo con mejor rendimiento. Al ajustar sus hiperparámetros, logramos mejorar su exactitud tanto en el conjunto de validación como en el conjunto de prueba.

Con una exactitud del **79.00%** en el conjunto de prueba, el modelo cumple con los requisitos del proyecto y puede ser utilizado para predecir con éxito el plan adecuado (Smart o Ultra) para los usuarios de Megaline basándose en su comportamiento de llamadas, mensajes y uso de datos.

Este modelo puede ser implementado por Megaline para optimizar sus recomendaciones de planes y mejorar la satisfacción del cliente.


## Prueba de cordura del modelo

Para verificar que nuestro modelo de **Bosque Aleatorio** esté funcionando correctamente y no presente errores fundamentales, realizamos una prueba de cordura. En esta prueba, comparamos el rendimiento del modelo con un método de predicción simple: devolver siempre el valor promedio del plan más común.

El objetivo de esta prueba es asegurarnos de que nuestro modelo predice mejor que un método trivial y que el desempeño no se debe simplemente al azar o a predicciones constantes.


In [9]:
# Prueba de cordura: predicción constante basada en el valor más común (mayoría)

# El plan más común en el conjunto de entrenamiento
most_common = target_train.mode()[0]

# Hacemos predicciones constantes basadas en el valor más común para todos los usuarios del conjunto de prueba
constant_predictions = [most_common] * len(target_test)

# Calculamos la exactitud de esta predicción constante
constant_accuracy = accuracy_score(target_test, constant_predictions)

# Imprimir los resultados
print(f"Exactitud de la predicción constante (prueba de cordura): {constant_accuracy:.4f}")


Exactitud de la predicción constante (prueba de cordura): 0.6843


## Prueba de cordura del modelo

Para asegurarnos de que el modelo de **Bosque Aleatorio** no esté haciendo predicciones triviales, realizamos una prueba de cordura. En esta prueba, comparamos el rendimiento del modelo con una predicción constante basada en el plan más común del conjunto de entrenamiento. El objetivo era verificar que el modelo predice mejor que una suposición simple.

### Resultados de la prueba de cordura:
- **Exactitud de la predicción constante**: 0.6843
- **Exactitud del Bosque Aleatorio**: 0.7900

### Análisis:
El modelo de **Bosque Aleatorio** supera significativamente la predicción constante, con un **10.57%** de mejora en la exactitud. Esto confirma que el modelo no está haciendo predicciones triviales y está utilizando los datos de comportamiento del cliente para predecir de manera más precisa si un cliente debería tener el plan **Smart** o **Ultra**.

El modelo ha pasado la prueba de cordura con éxito y puede ser implementado para predecir con confianza los planes de los usuarios de **Megaline**.
